In [10]:
from torchtext import legacy
import torch
SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = legacy.data.Field(tokenize ='spacy')
LABEL = legacy.data.LabelField(dtype=torch.float)

from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

import random
train_data, valid_data=train_data.split(random_state=random.seed(SEED))

TEXT.build_vocab(train_data, max_size = 25000, vectors = 'glove.6B.100d')
LABEL.build_vocab(train_data)
device = torch.device('cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((trian_data, valid_data, test_data),
                                                                          batch_size=BATCH_SIZE,
                                                                          device=device)


# word averaging


Please install SpaCy. See the docs at https://spacy.io for more information.


ModuleNotFoundError: No module named 'en'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functoinal as F

class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, pad_idx):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        self.linear = nn.Linear(embedding_size, output_size)
    
    def forward(self, text):
        embedded = self.embed(text) # [seg_len, batch_size, embedding_size]
        # embedded = embedded.transpose(1, 0) # [batch_size, seg_len, embedding_size]
        embedded = embedded.permuate(1, 0, 2)
        pooled F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze() # [batch_size, embedding_size]
        return self.linear(pooled)

In [11]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_SIZE = 100
OUTPUT_SIZE = 1
PAD_IDX = TEXT.vocab.stoi(TEXT.pad_token)

model = WordAVGModel(vocab_size = VOCAB_SIZE,
                    embedding_size = EMBEDDING_SIZE,
                    output_size = OUTPUT_SIZE,
                    pad_idx = PAD_IDX)

NameError: name 'TEXT' is not defined

In [12]:
pretrained_embedding = TEXT.vocab.vectors
mdoel.embed.weight.data.copy_(pretrained_embedding)


UNK_IDX = TEXT.vocab.stoi(TEXT.unk_token)
mdoel.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
mdoel.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

optimizer = torch.optim.Adam(model.parameters())
crit = nn.BCEWithLogitsLoss()

model = model.to(device)
crit = model.to(crit)



def binary_accuracy(preds, y):
    rounted_preds = torch.round(torch.sigmoid(preds))
    correct = (rounted_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

NameError: name 'TEXT' is not defined

In [13]:
def train(model, iterator, optimizer, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.train()
    for batch in iterator:
        preds = model(batch.text).squeeze() #[squeeze]
        loss = crit(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * len(batch_label)
        epoch_acc += acc.item() * len(batch_label)
        
        total_len += len(batch.label)
    return epoch_loss/len(iterator)

def evaluate(model, iterator, optimizer, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.eval()
    for batch in iterator:
        preds = model(batch.text).squeeze()
        loss = crit(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)        
        
        epoch_loss += loss.item() * len(batch_label)
        epoch_acc += acc.item() * len(batch_label)
        
        total_len += len(batch.label)
    model.train()
    return epoch_loss/len(iterator)

N_EPOCHS = 10
best_valid_acc = 0.
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, trian_iterator, optimizer, crit)
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit)
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'wordavg-model.pth')
    

    

NameError: name 'model' is not defined

In [ ]:
import spacy
nlp = spacy.load('en')

model.load_state_dict(torch.load('wordavg-model.pth'))
def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.toknizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    pred = torch.sigmoid(mode(tensor))
    return pred.item()